# Defining your Experimental Setup

This guide shows you how to create the object describing the experiment setup, which are needed for running the experiment workflows defined in the Applications Library.

## Create a `DeviceSetup`

Start by creating a `DeviceSetup` for your experimental setup by following the [Device Setup and Descriptor tutorial](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/tutorials/00_device_setup.html) in the LabOne Q Core documentation.

For demonstration purposes, here we create dummy `DeviceSetup` containing a SHFQC+ instrument, an HDAWG instrument, and a PQSC instrument, which are used to operate 3 tunable transmon qubits, labelled `q0, q1, q2`.

In [ ]:
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import (
    tunable_transmon_setup,
)

setup = tunable_transmon_setup(n_qubits=3)

The `setup` contains a logical signal group for each qubit labelled with the qubit UID, and each of these qubit signal-line group contains the following signal lines: `drive`, `drive_ef`, `measure`, `acquire`, `flux`, as shown below.

In [ ]:
qubit_signals = {
    quid: list(lsg.logical_signals) for quid, lsg in setup.logical_signal_groups.items()
}
qubit_signals

Let's inspect the connectivity between the instruments and the lines of the qubits:

In [ ]:
def get_physical_signal_name(quid, signal_name):
    logical_signal = setup.logical_signal_groups[quid].logical_signals[signal_name]
    return logical_signal.physical_channel.uid


connections = {
    quid: {
        sig_name: get_physical_signal_name(quid, sig_name) for sig_name in signals
    } for quid, signals in qubit_signals.items()
}

from pprint import pprint

pprint(connections)  # noqa: T203

We see that the three qubits are read out in parallel on the same quantum analyzer (QA) channel of the SHFQC instrument, and that their drive lines are controlled from individual signal generation (SG) channels of the SHFQC instrument. Finally, the flux lines of the qubits are controlled by individual HDAWG outputs. 

## Define qubits

We will show how to create qubit instances from the logical signal groups of the `DeviceSetup` defined above. Here, we use the `TunableTransmonQubit` class with corresponding `TunableTransmonQubitParameters`, but the procedure is the same for any other child class of LabOne Q `QuantumElements` class.

Note that, the qubit UIDs must match the names of the logical signal groups define in the `DeviceSetup` above, in this case `q0, q1, q2`.

In [ ]:
from laboneq_applications.qpu_types.tunable_transmon import (
    TunableTransmonQubit,
    TunableTransmonQubitParameters,
)

qubits = []
for i in range(3):
    q = TunableTransmonQubit.from_logical_signal_group(
        f"q{i}",
        setup.logical_signal_groups[f"q{i}"],
        parameters=TunableTransmonQubitParameters()
    )
    qubits.append(q)

The qubits are instantiated with identical default values of the `TunableTransmonQubitParameters` class. Let's see what they are:

In [ ]:
qubits[0].parameters

Adjust the values of the parameters to the ones for your quantum device. You can change the value of any of the parameters as shown below for the `drive_lo_frequency` parameter:

In [ ]:
qubits[0].parameters.drive_lo_frequency = 6e9

If you already the correct qubit parameters stored in an instance of `TunableTransmonQubitParameters` (for example, loaded from a file), you can directly pass them to the `parameters` argument of `TunableTransmonQubit.from_logical_signal_group`, and the qubits will be created with those parameters. 

## Define quantum operations

Next, we need to define the class of quantum operations implementing gates and operations on the qubits defined above. Here, we will use an instance of `TunableTransmonOperations` for the tunable transmons defined above.

In [ ]:
from laboneq_applications.qpu_types.tunable_transmon import TunableTransmonOperations

quantum_operations = TunableTransmonOperations()
quantum_operations.keys()

To learn more about quantum operations and how they are used to create quantum experiments from the qubit parameters, see the tutorial ...

## Define the `QPU`

Finally, we define the quantum processor (QPU) containing the qubits and the corresponding quantum operations.

The `qpu` contains the source of ground truth for an experiment and the best state of knowledge of the quantum system that is being operated. This means that the parameters of the qubits and any other parameters of the QPU define the configuration used by all the experiments in the Applications Library. 

In [ ]:
from laboneq.dsl.quantum import QPU

qpu = QPU(qubits=qubits, quantum_operations=quantum_operations)

## Loading From a File

The qubits and QPU can also be loaded back from `json` files saved by an experiment in the Applications Library. You just need the path to the file:

```
from laboneq import serializers

serializers.load(path_to_file)
```

## Optional: define a `QuantumPlatform`

Optionally, you can collect the `setup` and the `qpu` in an instance of `QuantumPlatform`.

In [ ]:
from laboneq.dsl.quantum import QuantumPlatform

qt_platform = QuantumPlatform(setup=setup, qpu=qpu)

## Demo `QuantumPlatform`

All of the objects used for demonstration purposes in this notebook can also be more conveniently obtained by instantiating a demo quantum platform provided by the Application Library. This demo platform is useful for quick prototyping in emulation mode. 

In [ ]:
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

demo_qt_platform = demo_platform(n_qubits=3)

In [ ]:
log_sig_groups = demo_qt_platform.setup.logical_signal_groups
qubit_signals = {
    quid: list(lsg.logical_signals) for quid, lsg in log_sig_groups.items()
}
qubit_signals

In [ ]:
demo_qt_platform.qpu.qubits[0].parameters

In [ ]:
demo_qt_platform.qpu.quantum_operations.keys()

## Connect to `Session`

Now let's connect to a LabOne Q `Session`. Here, we connect in emulation mode. When running on real hardware, connect using `do_emulation=False`.

In [ ]:
from laboneq.simple import Session

session = Session(setup)
session.connect(do_emulation=True)

Great! You have created everything you need to get started with the measurements. Now, on to experiments!